<a href="https://colab.research.google.com/github/silverstar0727/1day-1commit-challenge/blob/master/%ED%88%AC%EC%9E%90%EC%A7%80%ED%91%9C%EC%99%80_%EA%B0%80%EC%B9%98%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import requests

url = 'https://companyinfo.stock.naver.com/v1/company/c1040001.aspx?cmp_cd=005930'
html_text = requests.get(url).text

encparam = re.findall ("encparam: '(.*?)'", html_text)[0]
encparam

SSLError: ignored

In [0]:
#-*- coding: utf-8 -*-

# 네이버 파이낸스 투자지표 + 가치분석
# (c) 2018 FinaceData.KR

import re
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

def naver_invest_index(cmp_cd, rpt='5', frq='1', finGubun='IFRSL'):
    '''
    네이버 파이낸스로 부터 투자지표 읽어온다
    * cmp_cd (종목코드): '005930'
    * rpt (종류): '1'=수익성, '2'=성장성, '3'=안정성, '4'=활동성, '5'=가치분석 (기본값)
    * frq (기간): '0'=연간, '1'=분기(기본값)
    * finGubun (구분): 'MAIN'=주재무제표, 'IFRSS'=KIFRS별도, 'IFRSL'=IFRS연결(기본값), 'GAAPS'=GAAP개별, 'GAAPL'=GAAP연결
    '''
    # encparam 읽어오기
    url = 'https://companyinfo.stock.naver.com/v1/company/c1040001.aspx?cmp_cd=' + cmp_cd
    html_text = requests.get(url).text
    
    if not re.search("encparam: '(.*?)'", html_text):
        print('encparam not found') # encparam이 없는 경우 
        return None

    encparam = re.findall ("encparam: '(.*?)'", html_text)[0]

    # 투자지표 데이터 가져오기
    url_tmpl = 'https://companyinfo.stock.naver.com/v1/company/cF4002.aspx?' \
            'cmp_cd={cmp_cd}&frq={frq}&rpt={rpt}&finGubun={finGubun}&frqTyp={frq}&cn=&encparam={encparam}'
    url = url_tmpl.format(cmp_cd=cmp_cd, frq=frq, rpt=rpt, finGubun=finGubun, frqTyp=frq, encparam=encparam)

    # DataFrame 생성
    headers={'Referer': 'http://companyinfo.stock.naver.com'}
    jo = json.loads(requests.get(url, headers=headers).text)
    df = json_normalize(jo, 'DATA')

    # DATA1~DATA6 컬럼 이름 바꾸기 
    jo_yymm = jo['YYMM'][:6]
    date_str_list = []
    for yymm in jo_yymm:
        m = re.search('(\d{4}/\d{0,2}).*', yymm)
        date_str_list.append(m.group(1) if m else '') 
    data_n_list = ['DATA' + str(i) for i in range(1,7)]
    yymm_cols = zip(data_n_list, date_str_list)
    cols_map = dict(yymm_cols)
    df.rename(columns=cols_map, inplace=True)
    df['ACC_NM'] = df['ACC_NM'].str.strip().replace('[\.*\[\]]', '', regex=True)
    df = df.drop_duplicates(['ACCODE', 'ACC_NM'], keep='last')
    df.set_index(['ACCODE', 'ACC_NM'], inplace=True)
    df = df.iloc[:, 1:7] # 날짜 컬럼만 추출
    df = df.T # Transpose (컬럼, 인덱스 바꾸기)
    df.index = pd.to_datetime(df.index)
    df.index.name = '날짜'
    return df

In [2]:
# 삼양식품(003230), 수익성, 분기, IFRS연결

# 211300 매출총이익률
# 211000 영업이익률
# 211200 순이익률
# 211560 ROE
# 211650 ROA

# 코드만 사용할때는 Unique 하게 사용해야 한다
cols = ['211300', '211000', '211200', '211560', '211650' ] 

df = naver_invest_index('003230', rpt='1', frq='1', finGubun='IFRSL')
df[cols]

NameError: ignored